In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from sklearn import model_selection
from sklearn import linear_model
from sklearn import ensemble
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

%matplotlib inline

In [2]:
# This is where I parse out text
def clean_soup(soup):
    for tag in soup.find_all(['script', 'style','meta']):
        tag.decompose()   
    return soup.get_text()


def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""


In [3]:
import os

rootdir = '/home/roland/Workspace/Data/Procrastinating_HTML/'
data = pd.DataFrame(columns = ['activity', 'text', 'timestamp'])

URL_STRING = "__URL__: "
HTML_START = "<"
for subdir, dirs, files in os.walk(rootdir):
    i = 0
    for file in files:
        f = open(os.path.join(subdir, file))
        raw = f.read()
        url = ''
        
        if URL_STRING in raw:
            url = find_between(raw, URL_STRING, HTML_START)
            raw = raw.replace(url, '')
        
        # Below are character I am manually parsing out. There must exist a more efficient way to do this
        # but for now it runs fast enough.
        raw = raw.replace('\n', ' ')
        raw = raw.replace('\t', ' ')
        raw = raw.replace('\\n', ' ')
        raw = raw.replace('\\t', ' ')
        raw = raw.replace(URL_STRING, '')

        soup = BeautifulSoup(raw)
        data.set_value(i, 'text', clean_soup(soup))
        t = str(file).split('_')
        data.set_value(i, 'activity', t[0])
        timestamp = t[1].split('.')[0]
        data.set_value(i, 'timestamp', timestamp)
        data.set_value(i, 'url', url)
        i+=1
        f.close()


/home/roland/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/roland/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [4]:
data=data[data.text != ""]

In [5]:
data.timestamp = data.timestamp.str.replace("\(1\)","")
data.timestamp = data.timestamp.str.replace("T","-")

In [6]:
X = data[['text','url']]

X_train, X_test, y_train, y_test = model_selection.train_test_split(data, data.activity,test_size=0.33, random_state=43)

In [7]:
def evaluate_model(model, X_train = X_train, y_train = y_train, X_test = X_test, y_test = y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    
    cm = confusion_matrix(y_test, y_pred)
    cr = classification_report(y_test, y_pred)
    
    print(cm)
    print(cr)
    
    return accuracy

### Working with lemmatizing

In [8]:
# I am not using any of these libraries yet
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')

### Below is model building

In [9]:
# replacing any empty urls with empty strings
data.url = data.url.str.replace('NA', "")

In [10]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer

#instantiate our model, countvectorizing works th best
cvec = CountVectorizer(stop_words='english')

#fit the count vectorizer to the data. This 'teaches' the count vectorizer the dictionary.
#cvec.fit(data.text)

In [11]:
# # Vectorizing my text
# result_train = pd.DataFrame(data=cvec.transform(X_train.text).todense(), columns=cvec.vocabulary_)
# result_test = pd.DataFrame(data=cvec.transform(X_test.text).todense(), columns=cvec.vocabulary_)

# #NOTE TO SELF

# # Try using fuzzy matching on URLs. Try using the partial matching thing. Then label encode the ones that match well.

In [12]:
# Vectorizing my text
result_train = pd.DataFrame(data=cvec.fit_transform(X_train.text).todense(), columns = cvec.get_feature_names())
result_test = pd.DataFrame(data=cvec.transform(X_test.text).todense(), columns= cvec.get_feature_names())

#NOTE TO SELF

# Try using fuzzy matching on URLs. Try using the partial matching thing. Then label encode the ones that match well.

In [26]:
set(result_train.columns) - set(result_test.columns)

len(result_train.columns)

107457

#### Making my features here. 
I have my vectorized text which works really well, but there is additional informaiton that I can engineer that might be useful for prediction. Below is where I am going to work thourgh that process.

In [27]:
lm = linear_model.LogisticRegression(penalty='l1')
rf = ensemble.RandomForestClassifier()
# fitting my training data  and scoring with my test data
lm.fit(result_train,y_train)
lm.score(result_test, y_test)
evaluate_model(lm, X_train=result_train, X_test=result_test)

[[192   7]
 [  9 129]]
             precision    recall  f1-score   support

      procr       0.96      0.96      0.96       199
       work       0.95      0.93      0.94       138

avg / total       0.95      0.95      0.95       337



0.95252225519287836

In [28]:

predict = lm.predict(result_test)
actual = data.activity


In [59]:
features = result_train.columns[lm.coef_[0] != 0]
feature_values = lm.coef_[0][[lm.coef_[0] != 0]]
#coef = zip(result_test.columns, np.exp((lm.coef_[0])))
coef = list(zip(features, np.exp(feature_values)))
sorted(coef,key = lambda a: a[1], reverse=False)


[('append', 0.44929610912193518),
 ('old', 0.58786258208133035),
 ('twitter', 0.65663512952065128),
 ('playing', 0.66088982022858711),
 ('likes', 0.67352216976556345),
 ('2017', 0.6842875505291548),
 ('game', 0.69742697191050118),
 ('events', 0.72695334930494149),
 ('img', 0.75829224511122817),
 ('evil', 0.79115523707728774),
 ('don', 0.79753660518951641),
 ('movie', 0.81278686787357246),
 ('spell', 0.81636553824388636),
 ('damage', 0.82171754998691182),
 ('5e', 0.82916480976382945),
 ('watch', 0.83878894780645363),
 ('community', 0.87185627219275197),
 ('video', 0.87230001140819491),
 ('attack', 0.87298550188329382),
 ('vw', 0.87570121669833023),
 ('games', 0.89253273051489368),
 ('cookies', 0.89419362668445435),
 ('reddit', 0.90058401797536203),
 ('email', 0.91772173906004029),
 ('team', 0.9183408548699139),
 ('com', 0.92120008359321293),
 ('di', 0.92517556857286232),
 ('intelligence', 0.92693300801305734),
 ('00', 0.92910203712931161),
 ('frey', 0.93314319822640823),
 ('like', 0.935

In [61]:
len(coef)

112

In [17]:
# baseline
(y_train == 'procr').sum()/float(len(y_train))

0.58625730994152048

In [18]:
X_test['prob_work'] = lm.predict_proba(result_test)[:,1]*100

/home/roland/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
X_test['prob_work'] = X_test.prob_work.apply(int)
X_test['predict']  = lm.predict(result_test)

/home/roland/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/roland/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [62]:
X_test[X_test.activity != X_test.predict]

,activity,text,timestamp,url,prob_work,predict
1006,procr,"Guys, México has no net neutrality laws. This ...",2017-8-31-16-17-49,https://www.reddit.com/r/technology/comments/6...,99,work
443,work,How to make your tables less terrible : educat...,2017-8-25-17-8-11,https://www.reddit.com/r/educationalgifs/comme...,6,procr
861,work,pandas.DataFrame.append — pandas 0.20.3 docu...,2017-8-16-10-12-21,https://pandas.pydata.org/pandas-docs/stable/g...,3,procr
934,procr,[OC] snapchat heat map aligning with path of E...,2017-8-21-15-13-0,https://www.reddit.com/r/dataisbeautiful/comme...,96,work
385,procr,J.Crew - Shopping Bag ...,2017-8-4-11-42-54,,87,work
311,procr,streaming - How to watch Acestream / Sopcast ...,2017-8-26-21-41-33,https://askubuntu.com/questions/763364/how-to-...,99,work
893,work,City maps from Airbnb location ratings [OC] [2...,2017-8-9-10-7-28,,0,procr
364,procr,Python (genus) - Wikipedia ...,2017-8-5-3-23-47,,89,work
528,procr,"Each week for the last few months, I bought 3 ...",2017-8-24-1-23-46,https://www.reddit.com/r/dataisbeautiful/comme...,99,work
20,procr,dnd 5e - Can you use Minor Illusion to create...,2017-8-21-10-34-0,https://rpg.stackexchange.com/questions/77926/...,90,work


### Analyzing Count vectorized results

In [21]:
temp = result_train[list(X_train.activity =='procr')]

In [22]:
X_train['python_count']  = list(result_train['python'])

/home/roland/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [412]:
temp = X_train[X_train.python_count > 0].sort_values('python_count', ascending = False)
list(temp[temp.activity == 'procr'].iloc[0:1].text)

[" AnyDice            AnyDice Dice Probability Calculator   Articles      Stay Connected  Twitter,     Facebook,     Google+  Currently Playing     made by Jasper Flick     output 3d4   View       Data       ...calculating...   output 1 (92.00 / 9.42)#%240.00\xa0250.00\xa0260.00\xa0270.00\xa0280.00\xa0290.00\xa0300.00\xa0310.00\xa0320.00\xa0330.00\xa0340.00\xa0350.00\xa0360.00\xa0370.00\xa0380.00\xa0390.00\xa0400.00\xa0410.00\xa0420.00\xa0430.00\xa0440.00\xa0450.00\xa0460.00\xa0470.00\xa0480.00\xa0490.00\xa0500.00\xa0510.00\xa0520.00\xa0530.00\xa0540.00\xa0550.00\xa0560.00\xa0570.00\xa0580.00\xa0590.01\xa0600.01\xa0610.02\xa0620.02\xa0630.03\xa0640.05\xa0650.07\xa0660.09\xa0670.12\xa0680.16\xa0690.21\xa0700.28\xa0710.35\xa0720.45\xa0730.56\xa0740.69\xa0750.84\xa0761.01\xa0771.21\xa0781.42\xa0791.65\xa0801.90\xa0812.16\xa0822.42\xa0832.69\xa0842.96\xa0853.21\xa0863.45\xa0873.67\xa0883.85\xa0894.00\xa0904.12\xa0914.18\xa0924.21\xa0934.18\xa0944.12\xa0954.00\xa0963.85\xa0973.67\xa0983.45\

In [455]:
cvec.fit((temp[temp.activity == 'procr'].iloc[0:8].text))
new_temp = pd.DataFrame(data=cvec.transform(temp[temp.activity == 'procr'].iloc[0:8].text).todense(), columns = cvec.get_feature_names())

In [457]:
new_temp['anydice']

0    4
1    0
2    0
3    0
4    0
5    0
6    0
7    0
Name: anydice, dtype: int64

In [454]:
list(temp[temp.activity == 'procr'].iloc[0:1].text)

[" AnyDice            AnyDice Dice Probability Calculator   Articles      Stay Connected  Twitter,     Facebook,     Google+  Currently Playing     made by Jasper Flick     output 3d4   View       Data       ...calculating...   output 1 (92.00 / 9.42)#%240.00\xa0250.00\xa0260.00\xa0270.00\xa0280.00\xa0290.00\xa0300.00\xa0310.00\xa0320.00\xa0330.00\xa0340.00\xa0350.00\xa0360.00\xa0370.00\xa0380.00\xa0390.00\xa0400.00\xa0410.00\xa0420.00\xa0430.00\xa0440.00\xa0450.00\xa0460.00\xa0470.00\xa0480.00\xa0490.00\xa0500.00\xa0510.00\xa0520.00\xa0530.00\xa0540.00\xa0550.00\xa0560.00\xa0570.00\xa0580.00\xa0590.01\xa0600.01\xa0610.02\xa0620.02\xa0630.03\xa0640.05\xa0650.07\xa0660.09\xa0670.12\xa0680.16\xa0690.21\xa0700.28\xa0710.35\xa0720.45\xa0730.56\xa0740.69\xa0750.84\xa0761.01\xa0771.21\xa0781.42\xa0791.65\xa0801.90\xa0812.16\xa0822.42\xa0832.69\xa0842.96\xa0853.21\xa0863.45\xa0873.67\xa0883.85\xa0894.00\xa0904.12\xa0914.18\xa0924.21\xa0934.18\xa0944.12\xa0954.00\xa0963.85\xa0973.67\xa0983.45\